In [1]:
import json
import requests
import pandas as pd
import re
import string

In [18]:
# API로 데이터 불러오기
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'}
limit = 0
api_url = f'https://api.nasdaq.com/api/screener/stocks?tableonly=true&limit={limit}'
try:
    res = requests.get(api_url, headers=headers)
except Exception as e:
    print(e)

In [19]:
# deserialization
data = json.loads(res.text)

In [20]:
# 상장 종목 수 체크
num_of_listed_comp = data.get('data').get('totalrecords')

In [7]:
print(f"number of listed stocks : {num_of_listed_comp}")

number of listed stocks : 7795


In [8]:
# 상장 종목 리스트
stock_info = data['data']['table']['rows']
stock_info = pd.DataFrame(stock_info)
# 상장 종목 symbol이랑 name만 남김
stock_list = stock_info.iloc[:,:2]

In [11]:
# 정규표현식 설명
# \,| : comma(,) 매칭
# \s\(([\w].*)\)*| : 괄호로 쌓여있는 단어 매칭
# (\snew)?(\s[\w]*)?(\sstock|\sshare).*| : 'new 단어 stock', 'new 단어 share', '단어 stock', '단어 share' 뒤 모든 문자 매칭
# ((?<=inc\.)|(?<=inc))\s(\w|\W).*|((?<=ltd\.)|(?<=ltd))\s(\w|\W).*| : inc나 ltd 뒤 모든 문자 매칭
# (american.+[\w].share).* : 'american 단어 share' 뒤 모든 문자 매칭
# flag : 대소문자 무시

pattern = r'''
    \,|
    \s\(([\w].*)\)*|
    (\snew)?(\s[\w]*)?(\sstock|\sshare).*|
    ((?<=inc\.)|(?<=inc))\s(\w|\W).*|((?<=ltd\.)|
    (?<=ltd))\s(\w|\W).*|
    (american.+[\w].share+s?).*
    '''
regex = re.compile(pattern, flags=re.IGNORECASE)

In [ ]:
for index, row in stock_list.iterrows():
    # %가 포함 종목 제거
    if re.search(r'\%',row['name']):
        stock_list = stock_list.drop(index=index)
        continue

    # 필요 없는 문자 제거
    row['name'] = regex.sub('', row['name'])

In [ ]:
stock_list

In [24]:
# 상장 종목 업데이트
old_data = {"symbol":["AAPL","MSFT","GOOG"], "name":["Apple Inc.","Microsoft Corporation","Alphabet Inc."]}
new_data = {"symbol":["MSFT","GOOG","AMZN","FB"], "name":["Microsoft Corporation","Alphabet Inc.","Amazon.com Inc.","Facebook Inc."]}
old_df = pd.DataFrame(old_data)
new_df = pd.DataFrame(new_data)

In [25]:
old_df

,symbol,name
0,AAPL,Apple Inc.
1,MSFT,Microsoft Corporation
2,GOOG,Alphabet Inc.


In [26]:
new_df

,symbol,name
0,MSFT,Microsoft Corporation
1,GOOG,Alphabet Inc.
2,AMZN,Amazon.com Inc.
3,FB,Facebook Inc.


In [30]:
added_df = pd.concat([old_df, new_df]).drop_duplicates().reset_index(drop=True)
added_df

,symbol,name
0,AAPL,Apple Inc.
1,MSFT,Microsoft Corporation
2,GOOG,Alphabet Inc.
3,AMZN,Amazon.com Inc.
4,FB,Facebook Inc.
